In [1]:
import onto
import similarity
import importlib
import re
import time
import diffusion
import cross_validation

import pandas as pd
import numpy as np
import scipy
import cProfile

In [2]:
def reload_my_modules():
    """Re-import my modules into the IPython shell."""
    importlib.reload(onto)
    importlib.reload(similarity)
    importlib.reload(diffusion)
    importlib.reload(cross_validation)

In [3]:
# if you change module code, run this cell to reimport code into nb
reload_my_modules()

In [4]:
# load human go term annotations
fp = 'data/goa_human.gaf'
corpus = onto.Annotations(fp)
corpus.filter(keep_namespace="P")
print(len(corpus.annotations))

/home/xyz/Documents/CodeRepos/envs/ggid/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


166320


In [5]:
t0 = time.time()
# load Gene Ontology of terms
fp = 'data/go-basic.obo'
ontology = onto.GoGraph(obo_fp = fp)
ontology.parse_ontology()
# Calculate specificity of each term give then annotation corpus
term_specificity = ontology.calculate_term_specificity(corpus)
ontology.assign_term_specificity(term_specificity)
print(time.time() - t0)

7.04152774810791


/home/xyz/Documents/CodeRepos/ggid/onto.py:402: RuntimeWarning: divide by zero encountered in log10
  specificity = -1 * np.log10(full_count / full_count.sum())


In [6]:
# load list of human kinases
# these are the kinases we want to build the network for
kinases = pd.read_csv('data/list_of_human_kinases.csv')
kinases.head()

,Unnamed: 0,gene_symbol,gene_synonym,uniprot
0,0,AKT1,AKT1,P31749
1,1,AKT2,AKT2,P31751
2,2,AKT3,AKT3,Q9Y243
3,3,CDC42BPA,MRCKA,Q5VT25
4,4,CDC42BPB,MRCKB,Q9Y5S2


In [7]:
# set up similarity calculator
# we are building a network where kinases with similar go terms are connected

t0 = time.time()
kinase_similarity = similarity.Calculator(annotations = corpus,
                                    ontology=ontology, 
                                    proteins=kinases.gene_symbol)

# filter out misnamed and under annotated kinases
print("# kinases:", len(kinase_similarity.proteins))
filtered_out = kinase_similarity.filter_proteins(min_annotations=10)
print("# kinases, post-filter:", len(kinase_similarity.proteins))

#calculate similarity of go terms between each kinase pair
kinase_similarity.calculate_similarity()
print("similarity matrix for %d kinases constructed in %3.2f sec" % (len(kinase_similarity.proteins), time.time() - t0))

# kinases: 513
# kinases, post-filter: 327
similarity matrix for 327 kinases constructed in 48.19 sec


In [8]:
# load similarity scores and protein names into network container
network = similarity.Network(protein_similarity=kinase_similarity.protein_similarity,
                             proteins=kinase_similarity.proteins)

# the two lines below convert similarity matrix into a an adjacency matrix (ie network of 1s and 0s)
# we do this in the cross-validation block below and create several networks
#network.threshold_matrix(n=5)
#network.enforce_network_symmetry()

In [10]:
# do some edge num cross-validation

# positive set -- p53 kinases
# We are testing whether these proteins cluster in the kinase network 
# which they should because they phosphorylate the same protein, ie have similar go terms.
# If they cluster, then diffusion should predict left out positives in LOO validation.
p53_kin = ['CSNK2A1', 'CDK1', 'PRKDC', 'CDK2', 'MAPK8', 'CDK7',
           'CSNK1D', 'MAPK9', 'EIF2AK2', 'CHEK1', 'CHEK2', 'GSK3B',
           'MAPK1', 'PLK3','AURKA', 'TAF1', 'RPS6KA3', 'CDK9', 'CDK5',
           'DYRK2', 'HIPK2', 'IKBKB', 'TTK', 'AURKB', 'CSNK1A1', 'RPS6KA1']

p53_kin_exist = [p for p in p53_kin if p in network.proteins]
print("not in network:", [p for p in p53_kin if p not in network.proteins])

print("-----loo cross-validation-----")
print("keep top N edges per protein")
for edges in [1,2,3,5,10, 15, int(np.ceil(np.sqrt(len(network.proteins)))), 25, 30, 35, 50]:
    network.threshold_matrix(n = edges)
    network.enforce_network_symmetry()
    loo = cross_validation.LOOValitation(network, p53_kin_exist)
    res = loo.run_validation()
    y,x, auc = loo.get_roc()
    print("%d edges, auc = %1.3f" % (edges, auc))

not in network: ['TAF1']
-----loo cross-validation-----
keep top N edges per protein
1 edges, auc = 0.781
2 edges, auc = 0.819
3 edges, auc = 0.853
5 edges, auc = 0.845
10 edges, auc = 0.832
15 edges, auc = 0.850
19 edges, auc = 0.860
25 edges, auc = 0.894
30 edges, auc = 0.902
35 edges, auc = 0.892
50 edges, auc = 0.867


### Interpretation

The network seems to work. It clusters proteins with similar go terms. So we can use label diffusion to spread novel labels among connected proteins.

Most of the networks yeild LOO AUC .85 range, and the fewer edges you keep, the easier it is to interpret the results. So just go with 5 edges, and export the network for the front-end tool.

In [14]:
network.threshold_matrix(n=5)
network.enforce_network_symmetry()
network.save_as_pickle("network/kinase_matrix.pkl")